In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-01 09:36:45--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-01 09:36:45 (29.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
from openai import OpenAI

In [8]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key = 'ollama'
)

In [9]:
import os

In [14]:
llm("write that this is a test")

'This sentence simply serves as an instruction to write out the words "this is a test" on a piece of text, which appears in plain English. No further action or context beyond this statement seems necessary based on your initial prompt. If additional formatting for publication on platforms like social media were required, it might involve capitalization and punctuation adjustments as follows:\n\n```plaintext\nThis is a test! 😊 #TestMoment #WrittenCommunication Practice Check out the detailed step-bycurious individuals can learn from this simple exercise on effective communication strategies. Are you ready to try it? Let\'s begin our journey towards mastering clear expression today.\n```\n\nRemember, depending on your intent (formal vs. informal), additional style or tone changes might be necessary but not specified in the instruction provided.'

In [15]:
print(_)

This sentence simply serves as an instruction to write out the words "this is a test" on a piece of text, which appears in plain English. No further action or context beyond this statement seems necessary based on your initial prompt. If additional formatting for publication on platforms like social media were required, it might involve capitalization and punctuation adjustments as follows:

```plaintext
This is a test! 😊 #TestMoment #WrittenCommunication Practice Check out the detailed step-bycurious individuals can learn from this simple exercise on effective communication strategies. Are you ready to try it? Let's begin our journey towards mastering clear expression today.
```

Remember, depending on your intent (formal vs. informal), additional style or tone changes might be necessary but not specified in the instruction provided.
